In [ ]:
import pandas as pd
from sqlalchemy import create_engine

from django.conf import settings
from appcreator import creator

In [ ]:
SkosConcept.objects.all().count()

In [ ]:
sheet_id = settings.SHEET_ID_VOCABS

In [ ]:
df = creator.gsheet_to_df(sheet_id)

In [ ]:
dbc = settings.LEGACY_DB_CONNECTION
db_connection_str = f"mysql+pymysql://{dbc['USER']}:{dbc['PASSWORD']}@{dbc['HOST']}/{dbc['NAME']}"
db_connection = create_engine(db_connection_str)

In [ ]:
for i, row in df.iterrows():
    source_name = row['source_table']
    col_name = row['collection']
    pref_name_field = row['pref_label']
    legacy_id_field = row['legacy_id']
    query = f"SELECT * FROM {source_name}"
    skos_col, _ = SkosCollection.objects.get_or_create(
        name=col_name
    )
    try:
        df_data = pd.read_sql(query, con=db_connection)
        print(source_name)
        for i_data, row_data in df_data.iterrows():
            legacy_id = f"{col_name}__{float(row_data[legacy_id_field])}"
#             print(legacy_id)
            
            try:
                temp_item, _ = SkosConcept.objects.get_or_create(
                    legacy_id=legacy_id
                )
                temp_item.pref_label = row_data[pref_name_field]
                temp_item.save()
                temp_item.collection.add(skos_col)
            except Exception as e:
                print(f"storing {legacy_id} didn't work because of \n {e}")
    except Exception as e:
        df_data = False
        print(source_name, e)
        continue

In [ ]:
# SkosConcept.objects.all().delete()

In [ ]:
# SkosConcept.objects.all().count()